<a href="https://colab.research.google.com/github/AnalyticsKnight/yemoonsaBigdata/blob/dev/MPG_%EB%B6%84%EB%A5%98%ED%9A%8C%EA%B7%80_%EA%B9%80%EC%A3%BC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### mtcars: Motor Trend Car Road Tests
- https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv

In [31]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
# pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

##### 데이터 불러오기
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv")
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [32]:
df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin', 'name'],
      dtype='object')

1. 분류, Y = isUSA
- 분류 분석을 하기 위해 데이터를 7:3으로 split하기

In [75]:
# 불필요 변수
COL_DEL = ['name']
# 수치형 변수
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
# 범주형 변수 
COL_CAT = []
# 종속 변수
COL_Y = ['isUSA']

df_X = df[['name', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']]
df['isUSA'] = 0
df.loc[df.origin == 'usa', 'isUSA'] = 1
df_Y = df[['origin','isUSA']]

from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(df_X, df_Y, test_size=0.3)

X_raw = pd.concat([X_train_raw, X_test_raw]) #위 아래로 합치기,
# Scaling 및 Onehot encoding을 위한 데이터
train_raw =  pd.concat([X_train_raw, y_train_raw], axis=1) # 왼쪽 오른쪽으로 합치기
# raw를 지우는 데이터 처리를 위한 데이터

In [76]:
# X_train_raw.head()
# print(X_train_raw.columns)
# print(X_train_raw.dtypes)
print(X_train_raw.info()) # 결측치 확인
# X_train_raw.describe() #이상치 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 246 to 41
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    273 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 19.5+ KB
None


In [77]:
##### 불필요 변수 제외
X_train = X_train_raw.iloc[:, 1:]
y_train = y_train_raw.iloc[:, 1:]
X_test = X_test_raw.iloc[:, 1:]

In [78]:
##### 결측치 처리하기
from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )

In [79]:
##### 수치형 변수 - 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_raw[COL_NUM])
X_train[COL_NUM]=scaler.transform(X_train[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

In [80]:
##### 범주형 변수 - 그 다음 원핫 인코딩 - 여기서는 제외했으므로 사용하지 않음
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(handle_unknown = 'ignore')
# ohe.fit(X_train[COL_CAT])
# # numpy array로 변환
# X_train_res = ohe.transform(X_train[COL_CAT])
# X_test_res = ohe.transform(X_test[COL_CAT])

# # 다시 DF화 하기
# X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names())
# X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names())
# # print(X_train_ohe.head())

# X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
# X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

In [81]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [ ]:
##### 모델학습하기

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
# C 값이 작을 수록 규제가 커지고 일반화
modelLR = LogisticRegression(C=0.1, random_state=0)
modelLR.fit(X_tr, y_tr)

# KNN 분류
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr)

# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(n_estimators=100, max_depth=10)
modelRF.fit(X_tr, y_tr)

# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

# XGBoost
from xgboost import XGBClassifier
modelX = XGBClassifier(n_estimators=100, learning_rate=0.02, max_depth=10)
xgb_model = modelX.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='logloss',eval_set=[(X_val, y_val)])

# 이미 결정해버림
y_val_pred = modelLR.predict(X_val)
# [0일 확률, 1일 확률]확률값
y_val_pred_proba = modelLR.predict_proba(X_val)

In [87]:
##### 평가하기
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, classification_report, mean_squared_log_error

# confusion_matrix
# score = confusion_matrix(y_val.iloc[:,0], y_val_pred_proba)

# classification_report
# score = classification_report(y_val, y_val_pred_proba)

# mean_squared_log_error
# score = mean_squared_log_error(y_val, y_val_pred_proba)

# roc_auc_score
score = roc_auc_score(y_val, y_val_pred_proba[:, 1])

print( score )

0.9475218658892127


2. 회귀, Y = mpg
- 회귀 분석을 하기 위해 데이터를 7:3으로 split하기

In [140]:
# 불필요 변수
COL_DEL = ['name']
# 수치형 변수
COL_NUM = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
# 범주형 변수 
COL_CAT = ['origin']
# 종속 변수
COL_Y = ['mpg']

df_X = df[['name', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']]
df_Y = df[['mpg']]

from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(df_X, df_Y, test_size=0.3)

X_train_raw.reset_index(drop=True, inplace=True)
X_test_raw.reset_index(drop=True, inplace=True)
y_train_raw.reset_index(drop=True, inplace=True)
y_test_raw.reset_index(drop=True, inplace=True)

X_raw = pd.concat([X_train_raw, X_test_raw]) #위 아래로 합치기,
# Scaling 및 Onehot encoding을 위한 데이터
train_raw =  pd.concat([X_train_raw, y_train_raw], axis=1) # 왼쪽 오른쪽으로 합치기
# raw를 지우는 데이터 처리를 위한 데이터

In [141]:
# X_train_raw.head()
# print(X_train_raw.columns)
# print(X_train_raw.dtypes)
print(X_train_raw.info()) # 결측치 확인
# X_train_raw.describe() #이상치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   cylinders     278 non-null    int64  
 2   displacement  278 non-null    float64
 3   horsepower    275 non-null    float64
 4   weight        278 non-null    int64  
 5   acceleration  278 non-null    float64
 6   model_year    278 non-null    int64  
 7   origin        278 non-null    object 
dtypes: float64(3), int64(3), object(2)
memory usage: 17.5+ KB
None


In [148]:
##### 불필요 변수 제외
X_train = X_train_raw.iloc[:, 1:]
y_train = y_train_raw
X_test = X_test_raw.iloc[:, 1:]

In [149]:
##### 결측치 처리하기
from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )

In [150]:
##### 수치형 변수 - 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_raw[COL_NUM])
X_train[COL_NUM]=scaler.transform(X_train[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

In [151]:
##### 범주형 변수 - 그 다음 원핫 인코딩 - 여기서는 제외했으므로 사용하지 않음
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X_raw[COL_CAT])
# numpy array로 변환
X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

# 다시 DF화 하기
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names())
# print(X_train_ohe.head())

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

In [152]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

In [204]:
##### 모델학습하기

# sklearn 선형 회귀
from sklearn.linear_model import LinearRegression
# C 값이 작을 수록 규제가 커지고 일반화
modelLR = LinearRegression()
results = modelLR.fit(X_tr, y_tr)

# statsmodel 선형 회귀
import statsmodels.api as sm 

X_tr2 = sm.add_constant(X_tr)
model2 = sm.OLS(y_tr, X_tr2)
results = model2.fit()

# 이미 결정해버림
y_val_pred = modelLR.predict(X_val)

In [205]:
#### 회귀계수 해석
# 절편
print(modelLR.intercept_)
# 기울기
coef = pd.Series(data=modelLR.coef_[0], index= X_train_fin.columns)
print(coef.sort_values())

print(results.summary())

[24.65669681]
weight         -5.494804
x0_usa         -1.759041
cylinders      -0.606918
horsepower     -0.023547
acceleration    0.318217
x0_europe       0.494336
x0_japan        1.264705
displacement    1.520156
model_year      3.066423
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     112.0
Date:                Tue, 25 Jan 2022   Prob (F-statistic):           1.01e-66
Time:                        13:31:02   Log-Likelihood:                -512.73
No. Observations:                 194   AIC:                             1043.
Df Residuals:                     185   BIC:                             1073.
Df Model:                           8                                         
Covariance Type:            nonrobu

In [196]:
##### 평가하기
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)                 # 1. mse
rmse = mean_squared_error(y_val, y_val_pred, squared=False) # 2. rmse : np.sqrt(mse)도 가능

r2 = r2_score(y_val, y_val_pred)                            # 3. r2
score = modelLR.score(X_val, y_val)                         # 이 것 또한 R2

print(score)
print('MSE : {0:.3f} , RMSE : {1:.3F} '.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2))

0.8087522895870305
MSE : 11.154 , RMSE : 3.340 
Variance score : 0.809
